**참고문헌: 핸즈온 머신러닝(2판), 올레리앙 제롱 지음, 박해선 옮김, 15장 RNN과 CNN을 사용해 시퀀스 처리하기 + 참고문헌:  딥 러닝을 이용한 자연어 처리 입문, 8장- 순환 신경망**

# 케라스의 SimpleRNN과 LSTM 이해하기

케라스의 SimpleRNN과 LSTM을 이해해봅니다.

이 장에서는 RNN이면에 있는 기본적인 개념을 먼저 살펴보고 시간을 거슬러 역전파하여 네트워크를 훈련하는 방법을 알아봅니다. 그 다음 이 네트워크를 사용해 시계열 예측을 수행해봅니다. 그런 후에 RNN에 있는 두 가지 주요 난제를 살펴 보겠습니다.

- 불안정한 그레이디언트는 순환 드롭아웃과 순환 층 정규화를 포함한 여러 가지 기법을 사용해 감소시킬 수 있습니다.

- (매우) 제한적인 단기 기억은 LSTM과 GRU 셀을 사용해 확장할 수 있습니다.

RNN이 순차 데이터를 다룰 수 있는 유일한 신경망은 아닙니다. 작은 시퀀스의 경우 일반적인 밀집 네트워크가 처리할 수 있고 오디오 샘플이나 텍스트처럼 매우 긴 시퀀스라면 합성곱 신경망도 실제로 잘 작동할 수 있습니다. 이 두 가능성을 살펴본뒤 마지막에 WaveNet을 구현하겠습니다. 이 네트워크는 수만 개의 타임 스템을 가진 시퀀스를 다룰 수 있는 CNN구조입니다. 

## 15.1. 순환 뉴런과 순환층

순환 신경망은 피드포워드 신경망과 매우 비슷하짐나 뒤쪽으로 순환하는 연결도 있다는 점이 다릅니다 (그림 참고). 왼쪽의 그림처럼 입력을 받아 추력을 만들고 자신에게도 출력을 보내는 뉴런 하나로 구성된 가장 간단한 RNN을 살펴 봅시다.

<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-1.png">

각 타음 스템(time step) $t$ (또는 프레임[frame])마다 이 순환 뉴런(recurrent neuron)은 물론 $x_{(t)}$와 이전 타음 스텝의 추력인 $y_{(t-1)}$을 입력으로 받습니다. 첫번째 타임 스텝에서는 이전 추력이 없으므로 일반적으로 0으로 설정합니다. 이작은 네트워크를 오른쪽처럼 시간을 축으로 하여 표현할 수 있습니다. 이를 시간에 따라 네트워크를 펼쳤다(unrolling the network through time)고 말합니다 (동일한 뉴런을 타임 스텝마다 하나씩 표현한 것입니다).







순환 뉴런으로 된 층은 쉽게 만들 수 있습니다. 아래 그림처럼 타임 스텝 $t$마다 모든 뉴런은 입력 벡터 $x_{(t)}$와 이전 타임 스텝의 추력 벡터  $y_{(t-1)}$을 받습니다. 이제 입력과 출력이 모두 벡터가 됩니다(뉴런 이 하나일 때는 출력이 스칼라입니다) 

<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-2.png">


각 순환 뉴런은 두 벌의 가중치를 가집니다. 순환 층 전체의 추력 벡터는 익숙한 형태인 아래식과 같이 계산됩니다.


$
\mathbf{y}_{(t)} = \phi\left({\mathbf{W}_x}^T{\mathbf{x}_{(t)}} + {{\mathbf{W}_y}^T\mathbf{y}_{(t-1)}} + \mathbf{b} \right)
$

피드포워드 신경망처럼 타임스텝 $t$에서의 모든 입력을 행렬 $X_{(t)}$로 만들어 미니배치 전체에 대해 순환 층의 출력을 한 번에 계산할 수 있습니다.

$
\begin{split}
\mathbf{Y}_{(t)} & = \phi\left(\mathbf{X}_{(t)} \mathbf{W}_{x} + \mathbf{Y}_{(t-1)} \mathbf{W}_{y} + \mathbf{b} \right) \\
& = \phi\left(
\left[\mathbf{X}_{(t)} \quad \mathbf{Y}_{(t-1)} \right]
  \mathbf{W} + \mathbf{b} \right) \text{ with } \mathbf{W}=
\left[ \begin{matrix}
  \mathbf{W}_x\\
  \mathbf{W}_y
\end{matrix} \right]
\end{split}
$



- $\mathbf{Y}_{(t)}$는 타임 스텝 $t$에서 미니배치에 있는 각 샘플에 대한 층의 출력을 탐은 $m \times n_{neurons}$ 행렬입니다 ($m$은 미니배치에 있는 샘플 수 이고 $n_{neurons}$는 뉴런 수 입니다).

- $\mathbf{X}_{(t)}$ 모든 샘플의 입력값을 담은 $m \times n_{inputs}$ 행렬입니다($n_{inputs}$ 는 입력 특성 수 입니다).

- $\mathbf{W}_{(x)}$는 현재 타임 스텝의 입력에 대한 연결 가중치를 담은  $n_{inputs} \times n_{neurons}$ 행렬입니다.

- $\mathbf{W}_{(y)}$는 현재 타임 스텝의 입력에 대한 연결 가중치를 담은  $n_{neurons} \times n_{neurons}$ 행렬입니다.

- $\mathbf{b}$는 각 뉴런의 편향을 담은 $n_{neurons}$ 크기의 벡터입니다.

- 가중치 행렬 $\mathbf{W}_{(x)}$와 $\mathbf{W}_{(y)}$는 종종 $(n_{inputs} + n_{neurons}) \times n_{neurons}$ 크기의 가중치 행렬 $\mathbf{W}$ 하나로 연결합니다 [위 식을 참고] 

- $[\mathbf{X}_{(t)} \quad \mathbf{Y}_{(t-1)}]$표기는 행렬 $\mathbf{X}_{(t)} \quad \mathbf{Y}_{(t-1)}$을 수평적으로 연결한다는 것을 나타앱니다. 



## 15.1.1. 메모리 셀

타임 스텝 $t$에서 순환 뉴런의 출력은 이전 타임 스텝의 모든 입력에 대한 함수이므로 이를 일종의 메모리 형태라고 말할 수 있습니다. 타임 스텝에서 걸쳐서 어떤 상태를 보존하는 신경망의 구성 요소를 메모리셀(memory cell, 혹은 간단히 셀)이라고 합니다. 하나의 순환 뉴런 또는 순환 뉴런의 층은 짧은 패턴만 학습할 수 있는 매우 기본적인 셀입니다. 일반적으로 타임 스템 $t$에서의 셀의 상태  $\mathbf{h}_{(t)}$ (h는 히든(hidden)을 의미)는 그 타임 스텝의 입력과 이전 타임 스텝의 상태에 대한 함수입니다. 즉,  $\mathbf{h}_{(t)} = f(\mathbf{h}_{(t)}, \mathbf{x}_{(t)})$입니다. 타임 스텝 $t$에서의 출력  $\mathbf{y}_{(t)}$도 이전 상태와 현재 입력에 대한 함수입니다.

지금까지 이야기한 기본 셀의 경우 출력은 셀의 상태와 동일하지만, 아래 그림 처럼 더 복잡한 셀에서는 항상 그렇지 않습니다.


<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-3.png">


## 15.1.2. 입력과 출력 시퀀스


<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-4.png">


RNN은 입력 시퀀스를 받아 출력 시퀀스 만들 수 있습니다. 이는 시퀀스-투-시퀀스 네트워크(sequence-to-sequence network)로, 예를 들어 주식가격 같은 시계열 데이터를 예측하는데 유용합니다. 최근 N일치의 주식 가격을 주입하면 네트워크는 각 입력값도가 하루 앞선 가격을 출력해야합니다(즉, N-1일 전부터 내일까지).

또는 입력 시퀀스를 네트워크에 주입하고, 마지막을 제외한 모든 출력을 무시할 수 있습니다. 이는 시퀀스-투-벡터(sequence-to-vector)로 예를 들어 영화 리뷰에 있는 연속된 단어를 주입하면 네트워크는 감성 점수를 출력합니다 (예, -1 (싫다) 에서 +1 (좋다)).

반대로 각 타임 스텝에서 하나의 입력 벡터를 반복해서 네트워크에 주입하고, 하나의 시퀀스를 출력할 수 있습니다. 이는 벡터-투-시퀀스 네트워크(vector-to-sequence network)로, 예를 들어 이미지(또는 CNN의 출력)를 입력하여 이미지에 대한 캡션을 출력할 수 있습니다.

마지막으로 인코더(encoder)라 부르는 시퀀스-투-벡터 네트워크 뒤에 디코더(decodoer)라 부르는 벡터-투-시퀀스 네트워크를 연결할 수 있습니다. 예를 들어 한 언어의 문장을 다른 언어로 번역하는데 사용할 수 있습니다. 한 언어의 문장을 네트워크로에 주입하면 인코더는 이 문장을 하나의 벡터 표현으로 변환하고 , 그 다음에 디코더가 이 벡터를 다른 언어의 문장으로 디코딩합니다. 인코더-디코더라 부르는 이런 이중 단계 모델은 하나의 시퀀스-투-시퀀스 RNN을 사용하여 한 단어씩 번역하는 것보다 휠썬 더 잘 작동합니다. 

# 기본적인 RNN

RNN을 훈련하기 위한 기법은 타임 스텝으로 네트워크를 펼치고 보통의 역전파를 사용하는 것입니다. 이런 전략을 BPTT(backpropagation through time)라고 합니다.

<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-5.png">


보통의 역전파와 같이 첫 번째 정방향 패스가 펼쳐진 네트워크를 통과합니다 (파선 화살표로 표시). 그러면 비용 함수 $C[\mathbf{Y}_{(0)}, \mathbf{Y}_{(1)}, ..., \mathbf{Y}_{(T)}]$ (T는 최대 타임 스텝)를 사용하여 출력 시퀀스가 평가됩니다. 이 비용 함수는 위 그림에서 나타난 것처럼 일부 출력을 무시할 수 있습니다 (예를 들어, 시퀀스-투-벡터 RNN에서는 마지막을 제외한 모든 출력이 무시됩니다). 비용 함수의 그레이디언트는 펼쳐진 네트워크를 따라 역방향으로 전파됩니다 (실선 활살표로 표시). 결국 모델 파라미터는 BPTT동안 계산된 그레이디언트를 사용하여 업데이트됩니다. 그레이디언트가 마지막 출력뿐만 아니라 비용 함수를 사용한 모든 출력에서 역방향으로 전파됩니다. 예를 들어 위 그림에서 네트워크의 마지막 세개의 출력 $\mathbf{Y}_{(2)}, \mathbf{Y}_{(3)}, \mathbf{Y}_{(4)}$를 사용하여 비용 함수가 계산됩니다. 

따라서 그레이디언트는 이 세 개의 출력을 거쳐 흐르지만 $\mathbf{Y}_{(0)}$과 $ \mathbf{Y}_{(1)}$은 거치지 않습니다. 또한 각 타임 스텝마다 같은 매개변수  $\mathbf{W}$와 $\mathbf{b}$가 사용되기 때문에 역전파가 진행되면 모든 타임 스텝에 걸쳐 합산될 것입니다.

다행이도 tf.keras가 복잡한 모든 것을 처리해줍니다. 예제를 통해 알아보겠습니다.


케라스로 RNN 층을 추가하는 코드는 다음과 같습니다.

```
from tensorflow.keras.layers import SimpleRNN

model.add(SimpleRNN(hidden_units))

```

인자를 사용할 때를 보겠습니다.


```
# 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_units, input_shape=(timesteps, input_dim)))

# 다른 표기
model.add(SimpleRNN(hidden_units, input_length=M, input_dim=N))
```



hidden_units = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일. RNN의 용량(capacity)을 늘린다고 보면 되며, 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가진다.

timesteps = 입력 시퀀스의 길이(input_length)라고 표현하기도 함. 시점의 수.

input_dim = 입력의 크기.

<img src = "https://wikidocs.net/images/page/22886/rnn_image6between7.PNG">



RNN 층은 (batch_size, timesteps, input_dim) 크기의 3D 텐서를 입력으로 받습니다. batch_size는 한 번에 학습하는 데이터의 개수를 말합니다. 여기서부터는 텐서의 개념을 반드시 이해해야 하므로 벡터와 행렬 연산 챕터의 텐서 설명을 참고하시기 바랍니다. 다만, 이러한 표현은 사람이나 문헌에 따라서, 또는 풀고자 하는 문제에 따라서 종종 다르게 기재되는데 의 그림은 문제와 상황에 따라서 다르게 표현되는 입력 3D 텐서의 대표적인 표현들을 보여줍니다.

주의할 점은 위 코드는 출력층까지 포함한 인공 신경망 코드가 아니라 주로 은닉층으로 간주할 수 있는 하나의 RNN 층에 대한 코드입니다. 해당 코드가 리턴하는 결과값은 하나의 은닉 상태 또는 정의하기에 따라 여러 개의 시점의 은닉 상태 입니다. 아래의 그림은 만약 전결합층(Fully-connected layer)을 출력층으로 사용하였을 경우의 인공 신경망 그림과 은닉층까지만 표현한 그림의 차이를 보여줍니다.

<img src= "https://wikidocs.net/images/page/22886/rnn_image7_ver2.PNG">



그렇다면 RNN 층은 위에서 설명한 입력 3D 텐서를 입력받아서 어떻게 은닉 상태를 출력할까요? RNN 층은 사용자의 설정에 따라 두 가지 종류의 출력을 내보냅니다. 메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서를 리턴합니다. 하지만, 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴합니다. 이는 RNN 층의 return_sequences 매개 변수에 True를 설정하여 설정이 가능합니다. output_dim은 앞서 코드에서 정의한 hidden_units의 값으로 설정됩니다.

<img src= "https://wikidocs.net/images/page/22886/rnn_image8_ver2.PNG">


위의 그림은 time step=3일 때, return_sequences = True를 설정했을 때와 그렇지 않았을 때 어떤 차이가 있는지를 보여줍니다. return_sequences=True를 선택하면 메모리 셀이 모든 시점(time step)에 대해서 은닉 상태값을 출력하며, 별도 기재하지 않거나 return_sequences=False로 선택할 경우에는 메모리 셀은 하나의 은닉 상태값만을 출력합니다. 그리고 이 하나의 값은 마지막 시점(time step)의 메모리 셀의 은닉 상태값입니다.

마지막 은닉 상태만 전달하도록 하면 다 대 일(many-to-one) 문제를 풀 수 있고, 모든 시점의 은닉 상태를 전달하도록 하면, 다음층에 RNN 은닉층이 하나 더 있는 경우이거나 다 대 다(many-to-many) 문제를 풀 수 있습니다.

## 데이터셋 생성

3개의 단변량 시계열(univariate time series)을 만들어 보겠습니다.

In [ ]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   웨이브 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + 웨이브 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + 잡음
    return series[..., np.newaxis].astype(np.float32)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$", legend=True):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(n_steps, y, "bo", label="Target")
    if y_pred is not None:
        plt.plot(n_steps, y_pred, "rx", markersize=10, label="Prediction")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)
    plt.hlines(0, 0, 100, linewidth=1)
    plt.axis([0, n_steps + 1, -1, 1])
    if legend and (y or y_pred):
        plt.legend(fontsize=14, loc="upper left")

fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(12, 4))
for col in range(3):
    plt.sca(axes[col])
    plot_series(X_valid[col, :, 0], y_valid[col, 0],
                y_label=("$x(t)$" if col==0 else None),
                legend=(col == 0))
# save_fig("time_series_plot")
plt.show()

## 기준 성능 계산하기

단순한 예측 (마지막 관측값을 사용해 예측합니다):

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential

y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

In [ ]:
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

선형 예측:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
import matplotlib as mpl

def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss)) + 0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss)) + 1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.axis([1, 20, 0, 0.05])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)

plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

## 간단한 RNN 사용하기

하나의 뉴런으로 이루어진 하나의 층을 가집니다. 순환 신경망은 어떤 기링의 타임 스텝도 처리할 수 있기 때문에 입력 시퀀스의 길이를 지정할 필요가 없습니다. 따라서 첫 번째 입력 차원을 None으로 지정했습니다. 기본적으로 SimpleRNN층은 하이퍼볼릭 탄젠트 활성화 함수를 사용합니다. 이 층은 앞서 본 것과 완전히 동일하게 작동합니다.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

optimizer = keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

## 심층 RNN



RNN은 아래 그림처럼 셀을 여러 층을 쌓는 것이 일반적입니다. 이렇게 만든 것이 심층 RNN (deep RNN)입니다.

<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-7.png">

tf.keras로 심층 RNN을 구현하는 것을 매우 쉽습니다. 단순히 순환 층을 쌓으면 됩니다. 이 예에서는 세 개의 SimpleRNN 층을 사용합니다. 하지만 나중에 설명할 LSTM층이나 GRU층 같은 다른 종류의 순환 층을 추가할 수 있습니다.


CAUTION: 모든 순환 층에서 return_sequences=True로 설정해야 합니다(마지막 출력만 관심 대상이면 마지막 층에서는 설정하지 않습니다). 그렇지 않으면 (모든 타임 스텝에 대한 출력을 담은) 3D 배열이 아니라 (마지막 타임 스텝의 출력만 담음) 2D 배열이 출력되고 다음 순환 층이 3D 형태로 시퀀스를 받지 못하기 때문에 작동을 하지 못합니다.

보충자료: https://blog.naver.com/2000kgy/222603672877

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

hidden_state = model(X_train)
print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))



In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

두 번째 `SimpleRNN` 층은 마지막 출력만 반환합니다:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1) # 기본적으로 SimpleRNN은 tanh를 사용하는데 다른 활성화 함수를 사영하기 위해서 Dense로 바꿔서 사용도 가능합니다.
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

## 여러 타임 스텝 앞을 예측하기

지금까지는 다음 타임 스텝의 값만 예측했지만 타깃을 적절히 바꾸어 여러 타임 스텝 앞의 값을 손쉽게 예측할 수 있습니다(예를 들어, 1 스텝 앞이 아니라 10 스텝 앞의 값으로 타킷을 바꾸어 10 스텝 앞을 예측합니다). 하지만 다음 값 10개를 예측하고 싶다면 어떻게 해야할까요? 

첫 번째 방법은 이미 훈련된 모델을 사용하여 다음 값을 예측한 다음 이 값을 입력으로 추가하는 것입니다 (예측된 값이 실제로 발견된 것처럼 다룹니다). 이 모델을 사용해 다시 다음 값을 예측하는 식입니다. 다음 코드와 같습니다.

In [ ]:
np.random.seed(43) # 42는 훈련 세트에 있는 첫 번째 시리즈를 반환하기 때문에 다른 값으로 지정합니다

series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

Y_pred = X[:, n_steps:]

In [ ]:
Y_pred.shape

예상할 수 있듯이 다음 스텝에 대한 예측은 보통 더 미래의 타임 스텝에 대한 예측보다 정확합니다. 미래의 타임 스텝은 오차가 누적될 수 있기 때문입니다. 

In [ ]:
def plot_multiple_forecasts(X, Y, Y_pred):
    n_steps = X.shape[1]
    ahead = Y.shape[1]
    plot_series(X[0, :, 0])
    plt.plot(np.arange(n_steps, n_steps + ahead), Y[0, :, 0], "bo-", label="Actual")
    plt.plot(np.arange(n_steps, n_steps + ahead), Y_pred[0, :, 0], "rx-", label="Forecast", markersize=10)
    plt.axis([0, n_steps + ahead, -1, 1])
    plt.legend(fontsize=14)

plot_multiple_forecasts(X_new, Y_new, Y_pred)
#save_fig("forecast_ahead_plot")
plt.show()

이 모델을 사용해 다음 10개의 값을 예측해 보겠습니다. 먼저 아홉 개의 타임 스텝을 더 가진 시퀀스를 다시 생성해야 합니다.

In [ ]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

이제 하나씩 다음 10개의 값을 예측합니다:

In [ ]:
X = X_valid
for step_ahead in range(10):
    y_pred_one = model.predict(X)[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

Y_pred = X[:, n_steps:, 0]

In [ ]:
Y_pred.shape

In [ ]:
np.mean(keras.metrics.mean_squared_error(Y_valid, Y_pred))

이 성능을 단순한 예측이나 간단한 선형 모델과 비교해 보죠:

In [ ]:
Y_naive_pred = np.tile(X_valid[:, -1], 10) # 마지막 타임 스텝 값을 선택해 10번 반복합니다
np.mean(keras.metrics.mean_squared_error(Y_valid, Y_naive_pred))

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

이제 동시에 다음 10개의 값을 모두 예측하는 RNN을 만들어 보겠습니다:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, -10:, :]
Y_pred = model.predict(X_new)[..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)
plt.show()

이제 타임 스텝마다 다음 10 스텝을 예측하는 RNN을 만들어 보겠습니다. 즉 타임 스텝 0에서 49까지를 기반으로 타임 스텝 50에서 59를 예측하는 것이 아니라, 타임 스텝 0에서 타임 스텝 1에서 10까지 예측하고 그다음 타임 스텝 1에서 타임 스텝 2에서 11까지 예측합니다. 마지막 타임 스텝에서는 타임 스텝 50에서 59까지 예측합니다. 이 모델은 인과 모델입니다. 어떤 타임 스텝에서 예측을 만들 때 과거 타임 스텝만 볼 수 있습니다.

In [ ]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, 50:, :]
Y_pred = model.predict(X_new)[:, -1][..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)
plt.show()

# 배치 정규화(Batch Normalization)를 사용한 심층 RNN

긴 시퀀스로 RNN을 훈련하려면 많은 타임 스텝에 걸쳐 실행해야 하므로 펼친 RNN이 매우 깊은 네트워크가 됩니다. 보통 심층 신경망 처럼 그레이디언트 소실과 폭주 문제를 가질 수 있습니다. 훈련하는데 아주 오랜 시간이 걸리거나 훈련이 불안정할 수 있습니다. 또한 RNN이 긴 시퀀스를 처리할 때 입력의 첫 부분을 조금씩 잊어버릴 것입니다. 불안정한 그레이디언트 문제부터 시작해서 이 두 문제를 자세히 알아봅시다. 

불안정한 그레이디언트 문제를 완하하기 위해 심층 신경망에 사용했던 많은 기법(예, 좋은 가중치 초기화, 빠른 옵티마이저, 드롭아웃 등)을 RNN에서도 사용 할 수 있습니다. 그러나 수렴하지 않는 활성화 함수(예를 들어 ReLU)는 큰 도움이 되지 않습니다. 사실 이런 함수는 훈련한는 동안 RNN을 더 불안정하게 만들 수 있습니다. 

경사 하강법이 첫 번째 타임스텝에서 출력을 조금 증가시키는 방향으로 가중치를 업데이트한다고 가정해봅시다. 동일한 가중치가 모든 타임 스텝에서 사용되기 때문에 두 번째 타임 스텝의 출력도 조금 증가할 수 있고 세 번째 출력도 마찬가지입니다. 이런 식으로 출력이 폭주합니다. 수렴하지 않는 활성화 함수는 이를 막지 못합니다. 작은 학습율을 사용하여 이런 위험을 감소시킬 수 있찌만 간단히 하이퍼볼릭 탄젠트 같이 수렴하는 활성화 함수를 사용할 수 있습니다. 같은 방식으로 그레이디언트 자체도 폭주할 수 있습니다. 훈련이 불안정하다고 느껴지면 그레이디언트의 크기를 모니터링하고 그레이디언트 클리핑을 사용하는 것이 좋습니다.

배치 정규화는 심층 피드포워드 네크워크처럼 RNN과 효율적으로 사용할 수 없습니다. 사실 타임 스텝 사이에 사용할 수 없고 순환층 사이에만 가능합니다. 조금 더 정확히 말하면 기술적으로 메모리 셀에 배치 정규화 층을 추가하여 타임 스텝마다 (현재 타임 스텝의 입력과 이전 스텝의 은닉 상태 모두에) 적용할 수 있습니다. 하지만 입력이나 은닉상태의 실제 스케일과 이동에 상관없이 동일한 파라미터를 가진 배치 정규화가 타임 스텝마다 사용 될 것입니다. 

배치 정규화가 은닉 상태가 아니라 입력에 적용했을 때만 조금 도움이 된다는 것을 밝혔습니다(Cesar Laurent 2015). 다른 말로 하면 순환 층 안이 아니라 순환 층 사이(즉, 수직 방향)에 적용했을 때 없는 것보다 조금 낫습니다. 케라스에서는 간단히 각 순환 층 이전에 BatchNormalization 층을 추가합니다. 하지만 큰 기대를 안하는게 좋습니다.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.BatchNormalization(),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

# 층 정규화(Layer Normalization)를 사용한 심층 RNN

RNN에서 잘 맞는 다른 종류의 정규화는 층 정규화(Layer Normalization)입니다. 이 아이디어는 지미 바(Jimmy Lei Ba et al., 2016)등이 2016년 논문에서 소개했습니다. 배치 정규화와 매우 비슷하지만 배치 차원에 대한 정규화하는 대신 특성 차원에 대해 정규화합니다. 한 가지 장점은 샘플에 독립적으로 타임 스텝마다 동적으로 필요한 통계를 계산할 수 있다는 것입니다. 이는 훈련과 테스트에서 동일한 방식으로 작동한다는 것을 의미합니다(배치 정규화와는 다릅니다). 훈련 세트의 모든 샘플에 대한 특성 통계를 추정하기 위해 지수 이동 평균이 필요하지 않습니다. 배치 정규화와 마찬가지로 층 정규화는 입력마다 하나의 스케일과 이동 파라미터를 학습합니다. RNN에서 층 정규화는 일반적으로 입력과 은닉 상태의 선형 조합 직후에 사용됩니다.

참고자료: https://wingnim.tistory.com/92


In [ ]:
from tensorflow.keras.layers import LayerNormalization

In [ ]:
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
                                                          activation=None)
        self.layer_norm = LayerNormalization()
        self.activation = keras.activations.get(activation)
    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        if inputs is not None:
            batch_size = tf.shape(inputs)[0]
            dtype = inputs.dtype
        return [tf.zeros([batch_size, self.state_size], dtype=dtype)]
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
                     input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

# 사용자 정의 RNN 클래스 만들기

In [ ]:
class MyRNN(keras.layers.Layer):
    def __init__(self, cell, return_sequences=False, **kwargs):
        super().__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences
        self.get_initial_state = getattr(
            self.cell, "get_initial_state", self.fallback_initial_state)
    def fallback_initial_state(self, inputs):
        batch_size = tf.shape(inputs)[0]
        return [tf.zeros([batch_size, self.cell.state_size], dtype=inputs.dtype)]
    @tf.function
    def call(self, inputs):
        states = self.get_initial_state(inputs)
        shape = tf.shape(inputs)
        batch_size = shape[0]
        n_steps = shape[1]
        sequences = tf.TensorArray(
            inputs.dtype, size=(n_steps if self.return_sequences else 0))
        outputs = tf.zeros(shape=[batch_size, self.cell.output_size], dtype=inputs.dtype)
        for step in tf.range(n_steps):
            outputs, states = self.cell(inputs[:, step], states)
            if self.return_sequences:
                sequences = sequences.write(step, outputs)
        if self.return_sequences:
            return tf.transpose(sequences.stack(), [1, 0, 2])
        else:
            return outputs

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    MyRNN(LNSimpleRNNCell(20), return_sequences=True,
          input_shape=[None, 1]),
    MyRNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

## 양방향 순환 신경망(Bidirectional Recurrent Neural Network)

양방향 순환 신경망은 시점 t에서의 출력값을 예측할 때 이전 시점의 입력뿐만 아니라, 이후 시점의 입력 또한 예측에 기여할 수 있다는 아이디어에 기반합니다. 빈칸 채우기 문제에 비유하여 보겠습니다.



```
운동을 열심히 하는 것은 [        ]을 늘리는데 효과적이다.

1) 근육
2) 지방
3) 스트레스
```
'운동을 열심히 하는 것은 [ ]을 늘리는데 효과적이다.' 라는 문장에서 문맥 상으로 정답은 '근육'입니다. 위의 빈 칸 채우기 문제를 풀 때 이전에 나온 단어들만으로 빈 칸을 채우려고 시도해보면 정보가 부족합니다. '운동을 열심히 하는 것은' 까지만 주고 뒤의 단어들은 가린 채 빈 칸의 정답이 될 수 있는 세 개의 선택지 중 고르는 것은 뒤의 단어들까지 알고있는 상태보다 명백히 정답을 결정하기가 어렵습니다.

RNN이 풀고자 하는 문제 중에서는 과거 시점의 입력 뿐만 아니라 미래 시점의 입력에 힌트가 있는 경우도 많습니다. 그래서 이전과 이후의 시점 모두를 고려해서 현재 시점의 예측을 더욱 정확하게 할 수 있도록 고안된 것이 양방향 RNN입니다.

<img src= "https://wikidocs.net/images/page/22886/rnn_image5_ver2.PNG">

양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용합니다. 첫번째 메모리 셀은 앞에서 배운 것처럼 앞 시점의 은닉 상태(Forward States) 를 전달받아 현재의 은닉 상태를 계산합니다. 위의 그림에서는 주황색 메모리 셀에 해당됩니다. 두번째 메모리 셀은 앞에서 배운 것과는 다릅니다. 앞 시점의 은닉 상태가 아니라 뒤 시점의 은닉 상태(Backward States) 를 전달 받아 현재의 은닉 상태를 계산합니다. 입력 시퀀스를 반대 방향으로 읽는 것입니다. 위의 그림에서는 초록색 메모리 셀에 해당됩니다. 그리고 이 두 개의 값 모두가 현재 시점의 출력층에서 출력값을 예측하기 위해 사용됩니다.


```
from tensorflow.keras.layers import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))
```

양방향 RNN도 다수의 은닉층을 가질 수 있습니다. 아래의 그림은 양방향 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 양방향 순환 신경망의 모습을 보여줍니다.

<img src = "https://wikidocs.net/images/page/22886/rnn_image6_ver3.PNG">

다른 인공 신경망 모델들도 마찬가지이지만, 은닉층을 무조건 추가한다고 해서 모델의 성능이 좋아지는 것은 아닙니다. 은닉층을 추가하면 학습할 수 있는 양이 많아지지만 반대로 훈련 데이터 또한 많은 양이 필요합니다. 아래의 코드는 은닉층이 4개인 경우를 보여줍니다.



```
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
```



# 15.4.2 단기 기억 문제 해결하기

RNN을 거치면서 데이터가 변환되므로 일부 정보는 매 훈련 스텝 후 사라집니다. 어느 정도 시간이 지나면 RNN의 상태는 사실상 첫 번째 입력의 흔적을 가지고 있지 않습니다. 이는 심각한 문제가 될 수 있습니다. 

이런 문제를 해결하기 위해 장기 메모리를 가진 여러 종류의 셀이 연구됐습니다. 요즘에는 이런 셀들의 성능이 매우 좋아 기본 셀이 많이 사용되지 않습니다. 장기 메모리를 가진 셀에서 가장 인기 있는 LSTM 셀을 먼저 보겠습니다.

## 장단기 메모리 (long short-term memory, LSTM)

장단기 메모리 셀은 제프 호크라이터(Sepp Hochreiter)와 위르겐 슈미트후버(Jurgen Schmidhuber)가 1997년에 소개 했고, 알렉스 그레이브스(Alex Graves), 하심 삭(Hasim Sak), 보이치에흐 자렘바(Wojciech Zaremba) 등 여러 연구자 덕분에 수년간 점차 향상됐습니다. LSTM 셀을 블랙박스처럼 생각하면 기본 셀과 매우 비슷하게 사용할 수 있지만 성능은 훨씬 좋을 것 입니다. 즉, 훈련이 빠르게 수렴하고 데이터에 있는 장기간의 의존성을 감지할 것입니다. 케라스에서는 간단하게 SimpleRNN층 대신 LSTM층을 사용하면 됩니다.

LSTM 구조 그림은 아래와 같습니다.



<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-9.png">

$
\begin{split}
\mathbf{i}_{(t)}&=\sigma({\mathbf{W}_{xi}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hi}}^T \mathbf{h}_{(t-1)} + \mathbf{b}_i)\\
\mathbf{f}_{(t)}&=\sigma({\mathbf{W}_{xf}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hf}}^T \mathbf{h}_{(t-1)} + \mathbf{b}_f)\\
\mathbf{o}_{(t)}&=\sigma({\mathbf{W}_{xo}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{ho}}^T \mathbf{h}_{(t-1)} + \mathbf{b}_o)\\
\mathbf{g}_{(t)}&=\operatorname{tanh}({\mathbf{W}_{xg}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hg}}^T \mathbf{h}_{(t-1)} + \mathbf{b}_g)\\
\mathbf{c}_{(t)}&=\mathbf{f}_{(t)} \otimes \mathbf{c}_{(t-1)} \, + \, \mathbf{i}_{(t)} \otimes \mathbf{g}_{(t)}\\
\mathbf{y}_{(t)}&=\mathbf{h}_{(t)} = \mathbf{o}_{(t)} \otimes \operatorname{tanh}(\mathbf{c}_{(t)})
\end{split}
$


박스 안을 들여다보지 않는다면 LSTM 셀은 상태가 두개의 벡터 $\mathbf{h}_{(t)}$와 $\mathbf{c}_{(t)}$ (c는 셀(cell)을 의미)로 나뉜다는 것을 빼고는 정확히 일반 셀처럼 보입니다. $\mathbf{h}_{(t)}$를 단기 상태(short-term state), $\mathbf{c}_{(t)}$를 장기 상태(long-term state)라고 생각할 수 있습니다. 박스 안을 들여다 보면, 핵심 아이디어는 네트워크가 장기 상태에 저장할 것, 버릴 것, 그리고 읽어들을 것을 학습하는 것입니다. 

장기 기억 $\mathbf{c}_{(t-1)}$은 네트워크를 왼쪽에서 오른쪽으로 관통하면서, 삭제 게이트(forget gate)를 지나 일부 기억을 잃고, 그런 다음 덧셈 연산으로 새로운 기억 일부를 추가합니다(입력 게이트, input gate에서 선택한 기억을 추가합니다). 만들어진 $\mathbf{c}_{(t)}$는 다른 추가 변환 없이 바로 출력으로 보내집니다. 그래서 타임 스텝마다 일부 기억이 삭제되고 일부 기억이 추가됩니다. 또한 덧셈 연산 후 이 장기(long-term) 상태가 복사되 tanh 함수로 전달됩니다. 그런 다음 이결과는 출력 게이트(output gate)에 의해 걸러집니다. 이 단기 상태 $\mathbf{h}_{(t)}$를 만듭니다. 


이제 새로운 기억이 들어오는 곳과 게이트가 어떻게 작동하는지 살펴보겠습니다. 

먼저 현재 입력 벡터 $\mathbf{x}_{(t)}$와 이전의 단기 상태 $\mathbf{h}_{(t-1)}$이 네 개의 다른 완전 연결 층에 주입됩니다. 이 층은 모두 다른 목적을 가집니다. 

-  주 층은 $\mathbf{g}_{(t)}$를 출력하는 층입니다. 이 층은 현재 입력 $\mathbf{x}_{(t)}$와 이전의 (단기) 상태 $\mathbf{h}_{(t-1)}$을 분석하는 일반적인 역할을 담당합니다. 기본 셀에서는 이 층외에 다른 것은 없고 바로 $\mathbf{y}_{(t)}$와 $\mathbf{h}_{(t)}$로 출력됩니다. 반대로 LSTM에서는 이 층의 출력이 곧 바로 나가지 않고, 대신 장기 상태에 가장 중요한 부분이 저장됩니다 (나머지는 버림)

- 세 개의 다른 층은 게이트 제어기(gate controller)입니다. 이들은 로지스틱 활성화 함수를 사용하기 때문에 출력의 범위가 0에서 1 사이입니다. 그림에서 보듯이 이들의 출력은 원소별 곱셈연산으로 주입되어, 0을 출력하면 게이트를 닫고 1을 출력하면 게이트를 엽니다.

- 삭제 게이트($\mathbf{f}_{(t)}$가 제어함)는 장기 상태의 어느 부분이 삭제되어야 하는지 제어합니다

- 입력 게이터($\mathbf{i}_{(t)}$가 제어함)는 $\mathbf{g}_{(t)}$의 어느 부분이 장기 상태에 더해져야 하는지 제어합니다.

- 마지막으로 출력 게이트($\mathbf{o}_{(t)}$가 제어함)는 장기 상태의 어느 부분을 읽어서 이 타임 스텝의 $\mathbf{h}_{(t)}$와 $\mathbf{y}_{(t)}$로 출력해야 하는지 제어합니다.


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

In [ ]:
model.evaluate(X_valid, Y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, 50:, :]
Y_pred = model.predict(X_new)[:, -1][..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)
plt.show()

## 게이트 순환 유닛(gated recurrent unit, GRU) 셀

GRU 셀은 2014년 조경현(https://arxiv.org/abs/1406.1078) 등의 논문에서 제안됐습니다. 이 논문에서는 앞서 언급한 인코더-디코더 네트워크도 소개합니다.

<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-10.png">


GRU 셀은 LSTM 셀의 간소화된 버전이고 유사하게 작동하는 것처럼 보입니다. 간소화된 주요 내용은 다음과 같습니다.

- 두 상태 벡터가 하나의 벡터 $\mathbf{h}_{(t)}$로 합쳐졌습니다.

- 하나의 게이트 제어기 $\mathbf{z}_{(t)}$가 삭제 게이트와 입력 게이트를 모두 제어합니다. 게이트 제어기가 1을 출력하면 사겢 게이트가 열리고(= 1 ) 입력 게이트가 닫힙니다 (1 - 1 = 0). 게이트 제어기가 0을 출력하면 그 반대가 됩니다. 다시 말해 기억이 저장될 떄마다 저장될 위치가 먼저 삭제됩니다. 사실 이것 자체는 흔한 LSTM 셀의 변종입니다.

- 출력 게이트가 없습니다. 즉, 전체 상태 벡터가 매 타임 스텝마다 출력됩니다. 그러나 이전 상태의 어느 부분이 주 층($\mathbf{g}_{(t)}$)에 노출될지 제어하는 새로운 게이트 제어기  $\mathbf{r}_{(t)}$가 있습니다.

GRU 계산식


$
\begin{split}
\mathbf{z}_{(t)}&=\sigma({\mathbf{W}_{xz}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hz}}^T \mathbf{h}_{(t-1)}) \\
\mathbf{r}_{(t)}&=\sigma({\mathbf{W}_{xr}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hr}}^T \mathbf{h}_{(t-1)}) \\
\mathbf{g}_{(t)}&=\operatorname{tanh}\left({\mathbf{W}_{xg}}^T \mathbf{x}_{(t)} + {\mathbf{W}_{hg}}^T (\mathbf{r}_{(t)} \otimes \mathbf{h}_{(t-1)})\right) \\
\mathbf{h}_{(t)}&=(1-\mathbf{z}_{(t)}) \otimes \mathbf{h}_{(t-1)} + \mathbf{z}_{(t)} \otimes \mathbf{g}_{(t)}
\end{split}
$


케라스는 keras.layers.GRU층을 제공합니다. 이 층을 사용하려면 SimpleRNN이나 LSTM을 GRU로 바꾸면 됩니다.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

In [ ]:
model.evaluate(X_valid, Y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, 50:, :]
Y_pred = model.predict(X_new)[:, -1][..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)
plt.show()

## 1D 합성곱 층을 사용해 시퀀스 처리하기

LSTM과 GRU셀은 RNN 성공의 주역 중 하나입니다. 이 셀들이 단순한 RNN보다 훨씬 긴 시퀀스를 다룰 수 있지만 매우 제한적인 단기 기억을 가집니다. 100 타임 스텝 이상의 시퀀스에서 장기 패턴을 학습하는 데 어려움이 있습니다. 이 문제를 해결하는 한 가지 방법은 1D 합성 곱 층을 사용해 입력 시퀀스를 짧게 줄이는 것입니다.

2D 합성곱 층이 이미지에 대해 몇 개의 매우 작은 커널이 (또는 필터가) 슬라이밍 하며 (커널마다 하나씩) 2D 특성 맵을 만든다는 것을 보았습니다. 비슷하게 1D 합성곱 층이 몇 개의 커널을 시퀀스 위를 슬라이딩하며 커널마다 1D 특성 맵을 출력합니다. 각 커널은 매우 짧은 하나의 순차 패턴을 감지하도록 학습됩니다(커널 크기보다 길지 않습니다). 10 개의 커널을 사용하면 이층의 출력은 (모두 길이가 같은) 10개의 1차원 시퀀스로 구성됩니다. 또는 이 출력을 10차원 시퀀스 하나로 볼 수 있습니다. 이는 순환 층과 1D 합성곱 층 (또는 심지어 1D 풀링층)을 섞어서 신경망을 구성할 수 있다는 뜻 입니다.

```
1D conv layer with kernel size 4, stride 2, VALID padding:

              |-----2-----|     |-----5---...------|     |-----23----|
        |-----1-----|     |-----4-----|   ...      |-----22----|
  |-----0----|      |-----3-----|     |---...|-----21----|
X: 0  1  2  3  4  5  6  7  8  9  10 11 12 ... 42 43 44 45 46 47 48 49
Y: 1  2  3  4  5  6  7  8  9  10 11 12 13 ... 43 44 45 46 47 48 49 50
  /10 11 12 13 14 15 16 17 18 19 20 21 22 ... 52 53 54 55 56 57 58 59

Output:

X:     0/3   2/5   4/7   6/9   8/11 10/13 .../43 42/45 44/47 46/49
Y:     4/13  6/15  8/17 10/19 12/21 14/23 .../53 46/55 48/57 50/59
```

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train[:, 3::2], epochs=20,
                    validation_data=(X_valid, Y_valid[:, 3::2]))

## WaveNet

애런 밴 덴 오르드(Aaron van den Oord)와 다른 딥마인드 연구자들이 2016년 논문에서 WaveNet(https://arxiv.org/abs/1609.03499) 이라 부르는 구조를 소개했습니다. 이 네트워크는 층마다 (각 뉴런의 입력이 떨어져 있는 간격인) 팽창 비율(dilation rate)을 두 배로 늘리는 1D 합성곱 층을 쌓습니다. 첫 번째 합성곱 층이 한번에 2개의 타임 스텝만 바라봅니다. 다음 층은 4개의 타임 스텝을 보고(즉 수용장 길이가 4개의 타임 스텝에 됩니다). 다음 8개의 타임 스텝을 보는 식입니다. 이런 식으로 하위 층은 단기 패턴을 학습하고 상위 층은 장기 패턴을 학습합니다. 팽창 비율을 두배로 늘린 덕분에 네트워크는 아주 긴 시퀀스를 매우 효율적으로 처리할 수 있습니다.


<img src = "https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/RNN_15-11.png">

WaveNet 논문에서 저자들은 실제로 팽창 비율이 각각 1,2,4,8, ... , 256, 512인 합성곱 층 10개를 쌓았습니다. 그다음 동일한 층 10개를 따로 그룹지어 쌓았습니다. 이런 팽창 비율을 가진 합성곱 층 10개가 1,024 크기의 커널 한개로 이루어진 매우 효율적인 합성곱 층처럼 작동한다는 것을 보였습니다 (더 빠르고 강력하고 훨씬 적은 파라미터를 사용합니다). 이런 이유 때문에 이 블럭을 3개를 쌓았습니다. 각 층 이전의 팽창 비율과 동일한 개수의 0을 입력 시퀀스 왼쪽에 패딩으로 추가하여 네트워크를 통화하는 시퀀스 길이를 동일하게 만들었습니다. 다음은 앞과 동일한 시퀀스를 처리하는 간단한 WaveNet 구현입니다.

```
C2  /\ /\ /\ /\ /\ /\ /\ /\ /\ /\ /\ /\ /\.../\ /\ /\ /\ /\ /\
   \  /  \  /  \  /  \  /  \  /  \  /  \       /  \  /  \  /  \
     /    \      /    \      /    \                 /    \
C1  /\ /\ /\ /\ /\ /\ /\ /\ /\ /\ /\  /\ /.../\ /\ /\ /\ /\ /\ /\
X: 0  1  2  3  4  5  6  7  8  9  10 11 12 ... 43 44 45 46 47 48 49
Y: 1  2  3  4  5  6  7  8  9  10 11 12 13 ... 44 45 46 47 48 49 50
  /10 11 12 13 14 15 16 17 18 19 20 21 22 ... 53 54 55 56 57 58 59
```

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                  activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

다음은 논문에 정의된 원본 WaveNet입니다: ReLU 대신에 GatedActivationUnit과 스킵 연결을 사용합니다. 또한 점점 더 시퀀스가 짧아지는 것을 피하기 위해 왼쪽에 0으로 패딩합니다:

In [ ]:
class GatedActivationUnit(keras.layers.Layer):
    def __init__(self, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
    def call(self, inputs):
        n_filters = inputs.shape[-1] // 2
        linear_output = self.activation(inputs[..., :n_filters])
        gate = keras.activations.sigmoid(inputs[..., n_filters:])
        return self.activation(linear_output) * gate

In [ ]:
def wavenet_residual_block(inputs, n_filters, dilation_rate):
    z = keras.layers.Conv1D(2 * n_filters, kernel_size=2, padding="causal",
                            dilation_rate=dilation_rate)(inputs)
    z = GatedActivationUnit()(z)
    z = keras.layers.Conv1D(n_filters, kernel_size=1)(z)
    return keras.layers.Add()([z, inputs]), z

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

n_layers_per_block = 3 # 10 in the paper
n_blocks = 1 # 3 in the paper
n_filters = 32 # 128 in the paper
n_outputs = 10 # 256 in the paper

inputs = keras.layers.Input(shape=[None, 1])
z = keras.layers.Conv1D(n_filters, kernel_size=2, padding="causal")(inputs)
skip_to_last = []
for dilation_rate in [2**i for i in range(n_layers_per_block)] * n_blocks:
    z, skip = wavenet_residual_block(z, n_filters, dilation_rate)
    skip_to_last.append(skip)
z = keras.activations.relu(keras.layers.Add()(skip_to_last))
z = keras.layers.Conv1D(n_filters, kernel_size=1, activation="relu")(z)
Y_proba = keras.layers.Conv1D(n_outputs, kernel_size=1, activation="softmax")(z)

model = keras.models.Model(inputs=[inputs], outputs=[Y_proba])

In [ ]:
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=2,
                    validation_data=(X_valid, Y_valid))

이 장에서 RNN의 기초 사항을 살펴 보았고 RNN을 사용해 시퀀스(소위 시계열)을 처리했습니다. CNN을 포함하여 시퀀스를 처리하는 다른 방법도 알아 보았습니다. 다음 장에서는 RNN을 자연어 처리에 적용해 보겠습니다. 그리고 RNN에 대해 더 자세히 배워 보겠습니다(양방향 RNN, 상태가 있는 RNN과 상태가 없는 RNN, 인코더-디코더, 어텐션을 사용한 인코더-디코더). 또한 어텐션만 사용하는 구조인 트랜스포머도 살펴 보겠습니다.